In [5]:
import os
os.chdir('E:/GitHubProjects/educational/mathematical_modeling/')
import glob
from typing import Tuple, Literal
import argparse
from itertools import combinations

import numpy as np
import pandas as pd

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
def eval_corr_features(X, threshold=0.7, return_type:Literal['list','df']='list'):
    """
    Функция, которая признаки с нулевым (rij=0) и высоким (выше порога) значением корреляции. Использование функций необъодимо при оценке:
    
    1. анализ тесноты 
    2. оценка мультиколлинеарности
    """
    
    correlation_matrix = X.corr()
    correlated = [
            (i, j, correlation_matrix.loc[i, j]) 
            for i, j in combinations(correlation_matrix.columns, 2)
            if abs(correlation_matrix.loc[i, j]) >= threshold]
    
    if return_type == 'list':
        return correlated
    else:
        return pd.DataFrame(correlated, columns=['factor_1','factor_2','corr_value'])


In [10]:
data = pd.read_csv('lab2/data/final_dataset.txt', sep=';')

In [11]:
eval_corr_features(data)

[('growth_total', 'GDP', np.float64(-0.8111669757341613)),
 ('marriage_total', 'migration', np.float64(0.8718572332178485)),
 ('LE_total', 'migration', np.float64(-0.8013457152521691)),
 ('GDP', 'migration', np.float64(-0.7033657234073019))]

In [49]:
def feature_analysis(df):
    # Вычисляем корреляционную матрицу для всего DataFrame (включая таргет)
    corr_matrix = df.corr()
    
    # Разделим данные на предикторы (X) и таргет (y)
    X = df.iloc[:, :-1]  # Все столбцы, кроме последнего (предикторы)
    y = df.iloc[:, -1]   # Последний столбец (таргет)
    
    # Создаем список для результатов для X
    results_X = []
    for feature in X.columns:
        # Мультиколлинеарность для X: проверка на пороговое значение корреляции с другими предикторами
        multicollinearity_X = any(abs(corr_matrix[feature].drop(feature)) > 0.7)  # исключаем само-сравнение
        
        # Теснота для X: корреляция с таргетом
        target_correlation_X = corr_matrix[feature]['target']
        tightness_X = abs(target_correlation_X) < 0.01  # если корреляция с таргетом меньше 0.01
        
        # Отклик для X: определяем знак корреляции с таргетом
        response_X = None
    
        if target_correlation_X > 0:
            response_X = "+"
        elif target_correlation_X < 0:
            response_X = "-"
    
        # Добавляем информацию в результат
        results_X.append({
            "feature": feature,
            "multicollinearity": multicollinearity_X,
            "tightness": tightness_X,
            "response": response_X
        })
    
    # Результаты только для таргета в df
    target_results = []
    
    # Мультиколлинеарность для таргета: проверка на пороговое значение корреляции с предикторами
    for feature in X.columns:
        multicollinearity_df = abs(corr_matrix["target"][feature]) > 0.7  # проверка на мультиколлинеарность с предикторами
        target_results.append({
            "feature": f"target_{feature}",  # Формируем строку вида target_X1, target_X2 и т.д.
            "multicollinearity": multicollinearity_df,
            "tightness": None,  # Таргет не может иметь тесноту, так как это отклик
            "response": None
        })
    
    # Преобразуем список в DataFrame для X
    result_df_X = pd.DataFrame(results_X)
    result_df_df = pd.DataFrame(target_results)

    true_labels = result_df_X.loc[(result_df_X.multicollinearity==False)&(result_df_X.tightness==False)&(result_df_X.response=='+'), 'feature'].values
    
    # Вернем два результата и одну матрицу корреляции
    return result_df_X, result_df_df, corr_matrix, true_labels

# Пример использования
data = {
    'X1': [-312, 112, 258, 903, 233],
    'X2': [5, 4, 3, 2, 1],
    'X3': [20, 123, -1293, 2031, 123],
    'target': [1, 2, 3, 4, 5]
}

df = pd.DataFrame(data)

# Применяем функцию
result_df_X, result_df_df, corr_matrix, lbls = feature_analysis(df)

# Выводим результаты
print("Результаты для X:")
print(result_df_X)
print("\nРезультаты для df (только таргет):")
print(result_df_df)
print("\nКорреляционная матрица для df:")
print(corr_matrix)
print(lbls)

Результаты для X:
  feature  multicollinearity  tightness response
0      X1              False      False        +
1      X2               True      False        -
2      X3              False      False        +

Результаты для df (только таргет):
     feature  multicollinearity tightness response
0  target_X1              False      None     None
1  target_X2               True      None     None
2  target_X3              False      None     None

Корреляционная матрица для df:
              X1        X2        X3    target
X1      1.000000 -0.681852  0.626747  0.681852
X2     -0.681852  1.000000 -0.281845 -1.000000
X3      0.626747 -0.281845  1.000000  0.281845
target  0.681852 -1.000000  0.281845  1.000000
['X1' 'X3']


In [35]:
result_df

,feature,multicollinearity,tightness,response
0,X1,False,False,+
1,X2,True,False,-
2,X3,False,False,+
3,target,True,False,None


In [27]:
corr_matrix

,X1,X2,X3
X1,1.000000,-0.681852,0.626747
X2,-0.681852,1.000000,-0.281845
X3,0.626747,-0.281845,1.000000


In [51]:
data_dict = {
    'X1': [-312, 112, 258, 903, 233,-312, 112, 258, 903, 233,-312, 112, 258, 903, 233,-312, 112, 258, 903, 233],
    'X2': [5, 4, 3, 2, 1,5, 4, 3, 2, 1,5, 4, 3, 2, 1,5, 4, 3, 2, 1],
    'X3': [20, 123, -1293, 2031, 123,20, 123, -1293, 2031, 123,20, 123, -1293, 2031, 123,20, 123, -1293, 2031, 123],
    'target': [1, 2, 3, 4, 5,1, 2, 3, 4, 5,1, 2, 3, 4, 5,1, 2, 3, 4, 5]
}

data = pd.DataFrame(data_dict)

In [2]:
from sktime.datasets import load_shampoo_sales
y = load_shampoo_sales()